In [ ]:
# FlappyBIrd-DDQN Experiment
# 2020/08/11 SYC 

import models.ddqn as DDQN
import models.expStrategy.epsilonGreedy as EPSG
import envs.flappyBird as Game
import models.util as Util
import os
import logging
# To run tqdm on notebook, import tqdm.notebook
# from tqdm.notebook import tqdm
# Run on pure python
from tqdm import tqdm

# Config Logging format
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
# Config logging module to enable on notebook
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Block any pop-up windows
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Test GPU and show the available logical & physical GPUs
Util.test_gpu()

game = Game.FlappyBirdEnv()
NUM_STATE_FEATURES = game.get_num_state_features()
NUM_ACTIONS = game.get_num_actions()
BATCH_SIZE = 32
EPISODE_NUM = 20
PRINT_EVERY_EPISODE = 20

exp_stg = EPSG.EpsilonGreedy(0.1, NUM_ACTIONS)
agent = DDQN.Agent((NUM_STATE_FEATURES, ), NUM_ACTIONS, 1000, 0.9, 1e-5, exp_stg)

state = game.reset()
accum_reward = 0
bar = []
logging.info("Episode 1")
for episode in range(1, EPISODE_NUM + 1):
    
    if episode % PRINT_EVERY_EPISODE == 1:
        if episode > 1:
            bar.close()
            logging.info("Accumulated Reward: {} | Loss: {}".format(round(accum_reward / PRINT_EVERY_EPISODE), agent.get_metrics_loss()))
            logging.info("Episode {}".format(episode))
            agent.reset_metrics_loss()
            accum_reward = 0
        bar = tqdm(total = PRINT_EVERY_EPISODE)

    while not game.is_over():
        action = agent.select_action(state)
        state_prime, reward, is_done, info = game.act(action)
        # print(f'B State: {state}, Action: {action}, Reward: {reward}, State_Prime: {state_prime}')

        agent.add_buffer(state, action, reward, state_prime)
        is_update_target = agent.update(BATCH_SIZE)

        state = state_prime
        accum_reward += reward

    bar.update(1)        
    game.reset()

logging.info("Accumulated Reward: {} | Loss: {}".format(round(accum_reward / PRINT_EVERY_EPISODE), agent.get_metrics_loss()))
agent.reset_metrics_loss()
bar.close()

# Evaluate the model
agent.shutdown_explore()
agent.reset_metrics_loss()
# Reset Game
state = game.reset()
accum_reward = 0

while not game.is_over():
    action = agent.select_action(state)
    state_prime, reward, is_done, info = game.act(action)

    state = state_prime
    accum_reward += reward
    
logging.info("Evaluate")
logging.info("Accumulated Reward: {}".format(accum_reward))

In [3]:
# CartPole-REINFORCE Experiment
# 2020/08/11 SYC 

import models.REINFORCE as REINFORCE
import models.expStrategy.epsilonGreedy as EPSG
import envs.cartPole as cartPole
import models.util as Util
import logging
import matplotlib as plt
# To run tqdm on notebook, import tqdm.notebook
# from tqdm.notebook import tqdm
# Run on pure python
from tqdm import tqdm

# Config Logging format
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
# Config logging module to enable on notebook
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Test GPU and show the available logical & physical GPUs
Util.test_gpu()

env = cartPole.CartPoleEnv()
NUM_STATE_FEATURES = env.get_num_state_features()
NUM_ACTIONS = env.get_num_actions()
BATCH_SIZE = 32
EPISODE_NUM = 20
PRINT_EVERY_EPISODE = 20
LEARNING_RATE = 1e-4
REWARD_DISCOUNT = 0.9

exp_stg = EPSG.EpsilonGreedy(0.1, NUM_ACTIONS)
agent = REINFORCE.Agent((NUM_STATE_FEATURES, ), NUM_ACTIONS, REWARD_DISCOUNT, LEARNING_RATE, exp_stg)

state = env.reset()

accum_reward = 0
bar = []
logging.info("Episode 1")
for episode in range(1, EPISODE_NUM + 1):
    
    if episode % PRINT_EVERY_EPISODE == 1:
        if episode > 1:
            bar.close()
            logging.info("Accumulated Reward: {} | Loss: {}".format(round(accum_reward / PRINT_EVERY_EPISODE), agent.get_metrics_loss()))
            logging.info("Episode {}".format(episode))
            agent.reset_metrics_loss()
            accum_reward = 0
        bar = tqdm(total = PRINT_EVERY_EPISODE)

    while not env.is_over():
        # env.render()
        action = agent.select_action(state)
        state_prime, reward, is_done, info = env.act(action)

        agent.add_buffer(state, action, reward, state_prime)
        # print(f'State: {state}, Action: {action}, Reward: {reward}, State_Prime: {state_prime}')

        state = state_prime
        accum_reward += reward

    agent.update()
    agent.reset_buffer()

    bar.update(1)        
    env.reset()

bar.close()    
logging.info("Accumulated Reward: {} | Loss: {}".format(round(accum_reward / PRINT_EVERY_EPISODE), agent.get_metrics_loss()))
agent.reset_metrics_loss()


# Evaluate the model
agent.shutdown_explore()
agent.reset_metrics_loss()
# Reset Game
env_state = env.reset()
accum_reward = 0

while not env.is_over():
    # env.render()
    action = agent.select_action(state)
    state_prime, reward, is_done, info = env.act(action)

    state = state_prime
    accum_reward += reward

logging.info("Evaluate")
logging.info("Accumulated Reward: {}".format(accum_reward))

1 Physical GPUs, 1 Logical GPUs
2020-08-11 18:46:21,041 - Episode 1
100%|██████████| 20/20 [00:00<00:00, 22.85it/s]
2020-08-11 18:46:21,920 - Accumulated Reward: 13 | Loss: 0.1886632740497589
2020-08-11 18:46:21,935 - Evaluate
2020-08-11 18:46:21,937 - Accumulated Reward: 12.0
